In [1]:
import ast
import os
import random
import warnings

import autogluon.tabular
from IPython.display import display, HTML
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.linear_model
import sklearn.metrics
import sklearn.model_selection

pd.options.display.max_columns = 100
pd.options.display.max_colwidth = 200

warnings.filterwarnings("ignore", message="Can't initialize NVML")

In [2]:
DIRECTORY_DATA = 'data/model-autogluon'
os.makedirs(DIRECTORY_DATA, exist_ok=True)

# Preparing data

In [3]:
# FILENAME_DF_ALL_CLEAN = None
FILENAME_DF_ALL_CLEAN = 'data/20241230_173555/df_all_clean.csv'

In [4]:
SEPARATOR_COL = ': '


def col2parts(col: str) -> tuple[str, str]:
    parts = col.split(SEPARATOR_COL)
    if len(parts) == 1:
        return '', parts[0]
    assert len(parts) == 2
    return parts[0], parts[1]


if FILENAME_DF_ALL_CLEAN is not None:
    df_all = pd.read_csv(FILENAME_DF_ALL_CLEAN, header=list(range(2)), index_col=0)
    df_all.columns = [SEPARATOR_COL.join(col) for col in df_all.columns]
else:
    runnames = [
        '20241203_170129_all600',
        '20241213_104400_racing',
        '20241214_122216_racing_passhum',
    ]
    df_all = pd.concat(
        [pd.read_csv(f'data/{runname}/df_all.csv') 
         for runname in runnames],
        keys=runnames,
        names=['runname', 'row']
    )
    assert all(SEPARATOR_COL not in col for col in df_all.columns), df_all.columns
    
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1499 entries, map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities to map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing stops
Data columns (total 36 columns):
 #   Column                                                                             Non-Null Count  Dtype  
---  ------                                                                             --------------  -----  
 0   Violation type: Priority violation                                                 1499 non-null   bool   
 1   Violation type: Speed violation                                                    1499 non-null   bool   
 2   Violation type: Both                                                               1499 non-null   bool   
 3   Coordination strategy: Change of priorities                            

In [5]:
def series2values(series):
    dtype = series.dtype
    if dtype == 'bool':
        return series.astype('int').values
    if dtype in ('int64', 'float64'):
        return series.values
    if dtype == 'object':
        return series.astype('category').cat.codes
    raise TypeError(f'{dtype} is not supported')
    

if FILENAME_DF_ALL_CLEAN is not None:
    COLUMNS_NONSPLIT = {}
    COLUMNS_INPUT = {col: series2values 
                     for col in df_all.columns 
                     if col2parts(col)[0] != 'Output of simulation (execution)'
                     and not col2parts(col)[1].startswith('~ ')} 
    COLUMNS_OUTPUT = {col: series2values
                      for col in df_all.columns 
                      if col2parts(col)[0] == 'Output of simulation (execution)'
                      and not col2parts(col)[1].startswith('~ ')}
else:
    COLUMNS_NONSPLIT = {
        'Scenario ID': lambda series: series.values,
    }
    COLUMNS_INPUT = {
        'Coordination strategy': lambda series: series.astype('category').cat.codes.values,
        'isCanPassFirstHum': lambda series: series.astype('int').values,
        'isRacingThroughCrossroadAllowed': lambda series: series.astype('int').values,
        
        'Vehicle ID': lambda series: series.values,
        'Linearization C': lambda series: [
            series.apply(lambda x: -1 if x is None else x[i])
            for i in range(len(series.dropna().iloc[0]))
        ],   
    }
    COLUMNS_OUTPUT = {
        'traveled total, m': lambda series: series.values,
        'No. of completed missions': lambda series: series.values,
        'No. of collisions': lambda series: series.values,
        'No. of near-misses': lambda series: series.values,
    }

COLUMNS_ALL = {**COLUMNS_NONSPLIT, **COLUMNS_INPUT, **COLUMNS_OUTPUT}
COLUMNS_ALL

{'Violation type: Priority violation': <function __main__.series2values(series)>,
 'Violation type: Speed violation': <function __main__.series2values(series)>,
 'Violation type: Both': <function __main__.series2values(series)>,
 'Coordination strategy: Change of priorities': <function __main__.series2values(series)>,
 'Coordination strategy: Stops': <function __main__.series2values(series)>,
 'Coordination strategy: Rerouting': <function __main__.series2values(series)>,
 'Static map features: No. of OPs': <function __main__.series2values(series)>,
 'Static map features: Connectivity': <function __main__.series2values(series)>,
 'Output of simulation (planning): CSD score for MV (V0)': <function __main__.series2values(series)>,
 'Output of simulation (planning): CSD score for AV (V1)': <function __main__.series2values(series)>,
 'Output of simulation (planning): CSD score for AV (V2)': <function __main__.series2values(series)>,
 'Output of simulation (planning): CSD score for AV (V3)':

In [6]:
df_inout = df_all[list(COLUMNS_ALL)]
df_inout

,Violation type: Priority violation,Violation type: Speed violation,Violation type: Both,Coordination strategy: Change of priorities,Coordination strategy: Stops,Coordination strategy: Rerouting,Static map features: No. of OPs,Static map features: Connectivity,Output of simulation (planning): CSD score for MV (V0),Output of simulation (planning): CSD score for AV (V1),Output of simulation (planning): CSD score for AV (V2),Output of simulation (planning): CSD score for AV (V3),Output of simulation (planning): Mean CSD score for AVs (V1-V3),Output of simulation (planning): Mission length for MV (V0),Output of simulation (planning): Mission length for AV (V1),Output of simulation (planning): Mission length for AV (V2),Output of simulation (planning): Mission length for AV (V3),Output of simulation (planning): Total Mission length for AVs (V1-V3),Output of simulation (planning): Mean Mission length for AVs (V1-V3),Output of simulation (execution): No. of completed missions for AV (V1),Output of simulation (execution): No. of completed missions for AV (V2),Output of simulation (execution): No. of completed missions for AV (V3),Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): No. of collisions for AV (V1),Output of simulation (execution): No. of collisions for AV (V2),Output of simulation (execution): No. of collisions for AV (V3),Output of simulation (execution): Total No. of collisions for AVs (V1-V3),Output of simulation (execution): No. of near-misses for AV (V1),Output of simulation (execution): No. of near-misses for AV (V2),Output of simulation (execution): No. of near-misses for AV (V3),Output of simulation (execution): Total No. of near-misses for AVs (V1-V3)
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities",True,False,False,True,False,False,2,high,0.180,0.092,0.033,0.099,0.075,778,255,943,1517,2715,905.0,12,5,5,22,2,1,0,3,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing no",False,False,False,False,False,False,2,high,0.180,0.092,0.033,0.099,0.075,778,255,943,1517,2715,905.0,22,8,5,35,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing stops",True,False,False,False,True,False,2,high,0.180,0.092,0.033,0.099,0.075,778,255,943,1517,2715,905.0,5,2,5,12,2,0,0,2,0,1,0,1
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness with rerouting, forcing change of priorities",False,False,True,True,False,True,2,high,0.180,0.092,0.033,0.099,0.075,778,255,943,1517,2715,905.0,8,3,5,16,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness with rerouting, forcing no",False,True,False,False,False,True,2,high,0.180,0.092,0.033,0.099,0.075,778,255,943,1517,2715,905.0,21,8,5,34,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness no, forcing no",False,False,False,False,False,False,1,low,0.534,0.610,0.244,0.019,0.291,915,1115,1171,987,3273,1091.0,7,6,8,21,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness no, forcing stops",True,False,False,False,True,False,1,low,0.534,0.610,0.244,0.019,0.291,915,1115,1171,987,3273,1091.0,4,5,8,17,6,2,1,9,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing change of priorities",False,False,True,True,False,False,1,low,0.534,0.610,0.244,0.019,0.291,915,1115,1171,987,3273,1091.0,2,3,5,10,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/sce

In [7]:
def parse_tuple_string(tuple_string):
    if pd.isna(tuple_string):
        return None
    return ast.literal_eval(tuple_string)


def preprocess_inout(df_inout):
    df_inout = df_inout.copy()
    
    linearization_columns = [col for col in df_inout.columns if isinstance(col, str) and col.startswith('Linearization')]
    for col in linearization_columns:
        df_inout[col] = df_inout[col].apply(parse_tuple_string)
        
    dict_preprocessed = {}
    for col, series2data in COLUMNS_ALL.items():
        data = series2data(df_inout[col])
        if not isinstance(data, list):
            dict_preprocessed[col] = data
        else:
            for i, series in enumerate(data):
                dict_preprocessed[f'{col}#{i}'] = series
    
    return pd.DataFrame(dict_preprocessed)
    

df_preprocessed = preprocess_inout(df_inout)
df_preprocessed

,Violation type: Priority violation,Violation type: Speed violation,Violation type: Both,Coordination strategy: Change of priorities,Coordination strategy: Stops,Coordination strategy: Rerouting,Static map features: No. of OPs,Static map features: Connectivity,Output of simulation (planning): CSD score for MV (V0),Output of simulation (planning): CSD score for AV (V1),Output of simulation (planning): CSD score for AV (V2),Output of simulation (planning): CSD score for AV (V3),Output of simulation (planning): Mean CSD score for AVs (V1-V3),Output of simulation (planning): Mission length for MV (V0),Output of simulation (planning): Mission length for AV (V1),Output of simulation (planning): Mission length for AV (V2),Output of simulation (planning): Mission length for AV (V3),Output of simulation (planning): Total Mission length for AVs (V1-V3),Output of simulation (planning): Mean Mission length for AVs (V1-V3),Output of simulation (execution): No. of completed missions for AV (V1),Output of simulation (execution): No. of completed missions for AV (V2),Output of simulation (execution): No. of completed missions for AV (V3),Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): No. of collisions for AV (V1),Output of simulation (execution): No. of collisions for AV (V2),Output of simulation (execution): No. of collisions for AV (V3),Output of simulation (execution): Total No. of collisions for AVs (V1-V3),Output of simulation (execution): No. of near-misses for AV (V1),Output of simulation (execution): No. of near-misses for AV (V2),Output of simulation (execution): No. of near-misses for AV (V3),Output of simulation (execution): Total No. of near-misses for AVs (V1-V3)
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing change of priorities",1,0,0,1,0,0,2,0,0.180,0.092,0.033,0.099,0.075,778,255,943,1517,2715,905.0,12,5,5,22,2,1,0,3,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing no",0,0,0,0,0,0,2,0,0.180,0.092,0.033,0.099,0.075,778,255,943,1517,2715,905.0,22,8,5,35,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness no, forcing stops",1,0,0,0,1,0,2,0,0.180,0.092,0.033,0.099,0.075,778,255,943,1517,2715,905.0,5,2,5,12,2,0,0,2,0,1,0,1
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness with rerouting, forcing change of priorities",0,0,1,1,0,1,2,0,0.180,0.092,0.033,0.099,0.075,778,255,943,1517,2715,905.0,8,3,5,16,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-1.json, passhum 0, slowness with rerouting, forcing no",0,1,0,0,0,1,2,0,0.180,0.092,0.033,0.099,0.075,778,255,943,1517,2715,905.0,21,8,5,34,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness no, forcing no",0,0,0,0,0,0,1,1,0.534,0.610,0.244,0.019,0.291,915,1115,1171,987,3273,1091.0,7,6,8,21,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness no, forcing stops",1,0,0,0,1,0,1,1,0.534,0.610,0.244,0.019,0.291,915,1115,1171,987,3273,1091.0,4,5,8,17,6,2,1,9,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing change of priorities",0,0,1,1,0,0,1,1,0.534,0.610,0.244,0.019,0.291,915,1115,1171,987,3273,1091.0,2,3,5,10,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario9-9.json, passhum 0, slowness without rerouting, forcing no",0,1,0,0,0,0,1,1,0.534,0.610,0.244,0.019,0.291,915,1115,1171,987,3273,1091.0,3,6,8,17,0,0,0,0,0,0,0,0


In [8]:
def show(obj, title=None):
    if title is not None:
        display(HTML(f"<h3>{title}</h3>"))
    display(obj)


def shuffle_df(df):
    # Shuffle by Scenario ID
    unique_scenarios = df['Scenario ID'].unique()  # Get unique Scenario IDs
    shuffled_scenarios = pd.Series(unique_scenarios).sample(frac=1, random_state=1).tolist()  # Shuffle Scenario IDs
    
    # Reorder the dataframe by the shuffled Scenario IDs
    df_shuffled = pd.concat([df[df['Scenario ID'] == scenario] for scenario in shuffled_scenarios])
    
    # Reset index (optional)
    df_shuffled = df_shuffled.reset_index(drop=True)
    
    return df_shuffled


def split_train_test(df):
    test_size = 0.2
    random_state = 1
    
    if not COLUMNS_NONSPLIT:
        return sklearn.model_selection.train_test_split(df, test_size=test_size, random_state=random_state)
    
    gss = sklearn.model_selection.GroupShuffleSplit(n_splits=1, test_size=test_size, random_state=random_state)
    column, = list(COLUMNS_NONSPLIT)
    groups = df[column]
    
    # Split the data
    for train_idx, test_idx in gss.split(df, groups=groups):
        return shuffle_df(df.iloc[train_idx]), shuffle_df(df.iloc[test_idx])


df_train, df_test = split_train_test(df_preprocessed)
show(df_train, 'df_train')
show(df_test, 'df_test')

,Violation type: Priority violation,Violation type: Speed violation,Violation type: Both,Coordination strategy: Change of priorities,Coordination strategy: Stops,Coordination strategy: Rerouting,Static map features: No. of OPs,Static map features: Connectivity,Output of simulation (planning): CSD score for MV (V0),Output of simulation (planning): CSD score for AV (V1),Output of simulation (planning): CSD score for AV (V2),Output of simulation (planning): CSD score for AV (V3),Output of simulation (planning): Mean CSD score for AVs (V1-V3),Output of simulation (planning): Mission length for MV (V0),Output of simulation (planning): Mission length for AV (V1),Output of simulation (planning): Mission length for AV (V2),Output of simulation (planning): Mission length for AV (V3),Output of simulation (planning): Total Mission length for AVs (V1-V3),Output of simulation (planning): Mean Mission length for AVs (V1-V3),Output of simulation (execution): No. of completed missions for AV (V1),Output of simulation (execution): No. of completed missions for AV (V2),Output of simulation (execution): No. of completed missions for AV (V3),Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): No. of collisions for AV (V1),Output of simulation (execution): No. of collisions for AV (V2),Output of simulation (execution): No. of collisions for AV (V3),Output of simulation (execution): Total No. of collisions for AVs (V1-V3),Output of simulation (execution): No. of near-misses for AV (V1),Output of simulation (execution): No. of near-misses for AV (V2),Output of simulation (execution): No. of near-misses for AV (V3),Output of simulation (execution): Total No. of near-misses for AVs (V1-V3)
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-8.json, passhum 0, slowness no, forcing no",0,0,0,0,0,0,2,0,0.152,0.011,0.023,0.131,0.055,778,893,721,1205,2819,939.667,9,11,7,27,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-8.json, passhum 0, slowness no, forcing change of priorities",1,0,0,1,0,0,1,0,0.797,0.764,0.742,0.633,0.713,455,1047,1135,1791,3973,1324.333,5,4,4,13,0,0,8,8,0,7,0,7
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario4-3.json, passhum 0, slowness no, forcing stops",1,0,0,0,1,0,1,1,0.146,0.020,0.042,0.154,0.072,637,911,963,1425,3299,1099.667,8,6,4,18,0,0,6,6,0,1,1,2
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario6-10.json, passhum 0, slowness without rerouting, forcing stops",0,0,1,0,1,0,2,0,0.117,0.141,0.038,0.004,0.061,821,507,547,865,1919,639.667,11,13,9,33,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario3-10.json, passhum 0, slowness without rerouting, forcing stops",0,0,1,0,1,0,1,1,0.587,0.442,0.029,0.164,0.212,731,993,955,1563,3511,1170.333,3,8,2,13,1,0,1,2,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario7-9.json, passhum 0, slowness with rerouting, forcing stops",0,0,1,0,1,1,1,0,0.514,0.040,0.248,0.462,0.250,455,929,1643,1791,4363,1454.333,8,5,0,13,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario1-10.json, passhum 0, slowness no, forcing change of priorities",1,0,0,1,0,0,2,1,0.210,0.130,0.059,0.037,0.075,780,263,735,1107,2105,701.667,22,10,7,39,5,1,0,6,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario3-2.json, passhum 0, slowness without rerouting, forcing stops",0,0,1,0,1,0,1,1,0.743,0.904,0.754,0.049,0.569,731,993,671,935,2599,866.333,3,5,7,15,1,1,0,2,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario2-6.json, passhum 0, slowness no, forcing stops",1,0,0,0,1,0,1,0,0.287,0.036,0.773,0.769,0.526,587,741,1457,1507,3705,1235.000,9,2,2,13,1,5,1,7,0,0,0,0


,Violation type: Priority violation,Violation type: Speed violation,Violation type: Both,Coordination strategy: Change of priorities,Coordination strategy: Stops,Coordination strategy: Rerouting,Static map features: No. of OPs,Static map features: Connectivity,Output of simulation (planning): CSD score for MV (V0),Output of simulation (planning): CSD score for AV (V1),Output of simulation (planning): CSD score for AV (V2),Output of simulation (planning): CSD score for AV (V3),Output of simulation (planning): Mean CSD score for AVs (V1-V3),Output of simulation (planning): Mission length for MV (V0),Output of simulation (planning): Mission length for AV (V1),Output of simulation (planning): Mission length for AV (V2),Output of simulation (planning): Mission length for AV (V3),Output of simulation (planning): Total Mission length for AVs (V1-V3),Output of simulation (planning): Mean Mission length for AVs (V1-V3),Output of simulation (execution): No. of completed missions for AV (V1),Output of simulation (execution): No. of completed missions for AV (V2),Output of simulation (execution): No. of completed missions for AV (V3),Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): No. of collisions for AV (V1),Output of simulation (execution): No. of collisions for AV (V2),Output of simulation (execution): No. of collisions for AV (V3),Output of simulation (execution): Total No. of collisions for AVs (V1-V3),Output of simulation (execution): No. of near-misses for AV (V1),Output of simulation (execution): No. of near-misses for AV (V2),Output of simulation (execution): No. of near-misses for AV (V3),Output of simulation (execution): Total No. of near-misses for AVs (V1-V3)
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario10-1.json, passhum 0, slowness no, forcing no",0,0,0,0,0,0,2,0,0.203,0.140,0.045,0.096,0.093,871,479,453,1445,2377,792.333,15,15,6,36,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-8.json, passhum 0, slowness with rerouting, forcing change of priorities",0,0,1,1,0,1,2,0,0.152,0.011,0.023,0.131,0.055,778,893,721,1205,2819,939.667,9,10,5,24,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario5-2.json, passhum 0, slowness no, forcing change of priorities",1,0,0,1,0,0,1,1,0.549,0.118,0.286,0.534,0.312,628,969,1339,1873,4181,1393.667,7,5,4,16,0,4,4,8,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario3-6.json, passhum 0, slowness without rerouting, forcing no",0,1,0,0,0,0,1,0,0.469,0.512,0.116,0.026,0.218,731,995,937,957,2889,963.000,5,8,8,21,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-1.json, passhum 0, slowness no, forcing stops",1,0,0,0,1,0,2,1,0.301,0.176,0.042,0.131,0.116,872,505,453,1519,2477,825.667,3,4,5,12,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario8-7.json, passhum 0, slowness with rerouting, forcing no",0,1,0,0,0,1,1,0,0.707,0.915,0.708,0.098,0.574,868,711,1069,1471,3251,1083.667,4,4,4,12,0,0,0,0,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-6.json, passhum 0, slowness no, forcing stops",1,0,0,0,1,0,2,1,0.147,0.197,0.075,0.009,0.093,872,365,383,853,1601,533.667,16,16,9,41,4,0,2,6,2,0,0,2
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario3-8.json, passhum 0, slowness no, forcing stops",1,0,0,0,1,0,1,1,0.724,0.768,0.525,0.115,0.469,731,993,631,1451,3075,1025.000,1,3,2,6,4,1,0,5,0,0,0,0
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-10.json, passhum 0, slowness with rerouting, forcing change of priorities",0,0,1,1,0,1,2,0,0.088,0.069,0.028,0.014,0.037,778,255,721,1057,2033,677.667,21,10,7,38,0,0,0,0,0,0,0,0


# Logistic regression (as a baseline)

In [9]:
def split_df_to_X_y(df):
    columns_input_df = [col for col in df.columns
                        if (col if not isinstance(col, str) else col.split('#')[0]) in COLUMNS_INPUT]
    columns_output_df = list(COLUMNS_OUTPUT)
    assert set(COLUMNS_NONSPLIT) | set(columns_input_df) | set(columns_output_df) == set(df.columns)
    
    X = df[columns_input_df]
    y = df[columns_output_df]    
    return X, y


def run_regression(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    model = sklearn.linear_model.LinearRegression()
    model.fit(X_train, y_train)
    ndarray_predictions = model.predict(X_test)
    df_predictions = pd.DataFrame(ndarray_predictions, columns=y_test.columns)
    return df_predictions


df_predictions_regression = run_regression(df_train, df_test)
df_predictions_regression

,Output of simulation (execution): No. of completed missions for AV (V1),Output of simulation (execution): No. of completed missions for AV (V2),Output of simulation (execution): No. of completed missions for AV (V3),Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): No. of collisions for AV (V1),Output of simulation (execution): No. of collisions for AV (V2),Output of simulation (execution): No. of collisions for AV (V3),Output of simulation (execution): Total No. of collisions for AVs (V1-V3),Output of simulation (execution): No. of near-misses for AV (V1),Output of simulation (execution): No. of near-misses for AV (V2),Output of simulation (execution): No. of near-misses for AV (V3),Output of simulation (execution): Total No. of near-misses for AVs (V1-V3)
0,13.584160,11.866588,6.110373,31.561120,-0.038870,-0.405749,-0.318001,-0.762620,-0.066162,-0.084982,-0.050086,-0.201231
1,10.041086,9.550762,6.289827,25.881676,-0.156431,0.188733,0.383331,0.415633,0.168910,0.253336,-0.024295,0.397950
2,6.152065,4.053624,1.402782,11.608472,1.330424,1.025058,1.746001,4.101483,0.177736,0.234788,0.036241,0.448766
3,4.871056,6.351524,6.793021,18.015600,0.302273,-0.086925,0.071101,0.286450,-0.026258,-0.017747,0.026100,-0.017905
4,10.967487,9.943957,3.387107,24.298551,1.903121,0.679726,0.156430,2.739276,0.344737,0.248707,0.060353,0.653796
...,...,...,...,...,...,...,...,...,...,...,...,...
295,6.200005,4.085038,4.495208,14.780251,0.927933,0.208028,-0.040611,1.095350,0.214446,0.222988,0.008435,0.445868
296,11.687327,9.852621,6.760653,28.300602,1.901396,0.784896,0.035611,2.721902,0.310125,0.070320,0.083568,0.464014
297,1.788124,5.403498,2.225122,9.416744,2.294553,0.818835,0.464293,3.577680,0.397424,0.389105,0.075607,0.862136
298,14.465753,8.912797,7.313466,30.692016,0.076112,0.299425,0.450152,0.825689,0.093800,0.035245,-0.009083,0.119962


In [10]:
def save_and_show(fig, basename):  # to avoid inlining large image data into the notebook file
    filename = f'{DIRECTORY_DATA}/{basename}-{random.random()}.png'
    fig.savefig(filename)
    
    # The `random` is because of https://stackoverflow.com/a/43640705.
    display(HTML(f'<img src="{filename}?{random.random()}" alt="{basename}" />'))
    
    plt.close(fig)
    
    return filename


def evaluate_and_plot_column(df_test, df_predictions, column):
    y_test_column = df_test[column]
    predictions_column = df_predictions[column]
    
    r2 = sklearn.metrics.r2_score(y_test_column, predictions_column)
    name = col2parts(column)[1]
    print(f"{name}:")
    print(f"- R^2 Score: {r2}")

    # Plot results for each output column
    fig = plt.figure(figsize=(10, 6))
    plt.scatter(y_test_column, predictions_column, color='blue', alpha=0.5)
    plt.plot([y_test_column.min(), y_test_column.max()], [y_test_column.min(), y_test_column.max()], 'k--', lw=2)
    plt.xlabel('Actual Values')
    plt.ylabel('Predicted Values')
    plt.title(f'Actual vs Predicted Values for {name}')
    plt.grid(True)
    save_and_show(fig, f'Actual_vs_Predicted_Values_{name}')


def evaluate_and_plot_all_columns(df_test, df_predictions):
    for column in COLUMNS_OUTPUT:
        evaluate_and_plot_column(df_test, df_predictions, column)
        
        
evaluate_and_plot_all_columns(df_test, df_predictions_regression)

No. of completed missions for AV (V1):
- R^2 Score: 0.7125586390649102


No. of completed missions for AV (V2):
- R^2 Score: 0.5446412367153544


No. of completed missions for AV (V3):
- R^2 Score: 0.7306488914619292


Total No. of completed missions for AVs (V1-V3):
- R^2 Score: 0.6375117344630747


No. of collisions for AV (V1):
- R^2 Score: 0.40936671376104283


...

Total No. of collisions for AVs (V1-V3):
- R^2 Score: 0.344311170692268


No. of near-misses for AV (V1):
- R^2 Score: 0.09710818586433001


No. of near-misses for AV (V2):
- R^2 Score: 0.06104272967216251


No. of near-misses for AV (V3):
- R^2 Score: 0.0467726452527909


Total No. of near-misses for AVs (V1-V3):
- R^2 Score: 0.08515380204115597


# AutoGluon

In [11]:
def run_autogluon(df_train, df_test):
    X_train, y_train = split_df_to_X_y(df_train)
    X_test, y_test = split_df_to_X_y(df_test)
    
    # Train AutoGluon models
    predictors = []
    df_predictions = pd.DataFrame()
    for column in COLUMNS_OUTPUT:
        print(f'{column=}:')
        df_train_predictor = pd.concat([X_train, y_train[[column]]], axis=1)
        
        predictor = autogluon.tabular.TabularPredictor(
            label=column, 
            eval_metric='r2', 
            problem_type='regression',
        ).fit(
            df_train_predictor,
            presets='medium',
            hyperparameters={
                'GBM': {},       # LightGBM (TODO: something like `GBMLarge`)
                'XGB': {},       # XGBoost
                'RF': {},        # Random Forest
                'XT': {},        # Extra Trees
                # 'CAT': {},      # CatBoost, omitted if slow
                # 'NN': {},       # Neural net, if you want it
                # 'LR': {},       # Linear model
                # 'KNN': {},      # K-Nearest Neighbors
            },
        )
        predictors.append(predictor)
        
        df_predictions[column] = predictor.predict(X_test)

        # Leaderboard - Display a table of different models and their performance
        df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        leaderboard = predictor.leaderboard(df_test_predictor, silent=True)
        show(leaderboard, f'Leaderboard for {column}')
        
        # Feature importance on training data
        # show(
        #     predictor.feature_importance(df_train_predictor),
        #     'feature_importance(df_train_predictor)'
        # )
        # 
        # # Feature importance on test data
        # show(
        #     predictor.feature_importance(df_test_predictor),
        #     'feature_importance(df_test_predictor)'
        # )
        # Example: SHAP values for a specific model
        # shap_values = predictor.get_model_shap_values(df_test_predictor, model='LightGBM')
        # show(shap_values, 'shap_values')  # SHAP values for each feature and each prediction
        
    return predictors, df_predictions


predictors, df_predictions_autogluon = run_autogluon(df_train, df_test)
df_predictions_autogluon

No path specified. Models will be saved in: "AutogluonModels/ag-20250109_132619"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #135~20.04.1-Ubuntu SMP Mon Oct 7 13:56:22 UTC 2024
CPU Count:          16
Memory Avail:       3.70 GB / 31.09 GB (11.9%)
Disk Space Avail:   297.67 GB / 693.60 GB (42.9%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250109_132619"
Train Data Rows:    1199
Train Data Columns: 19
Label Column:       Output of simulation (execution): No. of completed missions for AV (V1)
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memor

column='Output of simulation (execution): No. of completed missions for AV (V1)':


User-specified model hyperparameters to be fit:
{
	'GBM': [{}],
	'XGB': [{}],
	'RF': [{}],
	'XT': [{}],
}
Fitting 4 L1 models, fit_strategy="sequential" ...
Fitting model: LightGBM ...
/home/olga/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:905: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  r = torch._C._cuda_getDeviceCount() if nvml_count < 0 else nvml_count
	0.8155	 = Validation score   (r2)
	3.89s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForest ...
	0.8086	 = Validation score   (r2)
	0.49s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: ExtraTre

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTrees,0.849826,0.835688,r2,0.056722,0.034693,0.381312,0.056722,0.034693,0.381312,1,True,3
1,WeightedEnsemble_L2,0.838735,0.845089,r2,0.072166,0.038718,1.328755,0.001926,0.000683,0.050108,2,True,5
2,RandomForest,0.836897,0.808619,r2,0.049459,0.034859,0.485480,0.049459,0.034859,0.485480,1,True,2
3,LightGBM,0.823602,0.815486,r2,0.005349,0.001831,3.894964,0.005349,0.001831,3.894964,1,True,1
4,XGBoost,0.820449,0.840546,r2,0.013518,0.003341,0.897335,0.013518,0.003341,0.897335,1,True,4


No path specified. Models will be saved in: "AutogluonModels/ag-20250109_132625"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #135~20.04.1-Ubuntu SMP Mon Oct 7 13:56:22 UTC 2024
CPU Count:          16
Memory Avail:       3.35 GB / 31.09 GB (10.8%)
Disk Space Avail:   297.64 GB / 693.60 GB (42.9%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250109_132625"
Train Data Rows:    1199
Train Data Columns: 19
Label Column:       Output of simulation (execution): No. of completed missions for AV (V2)
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memor

...

No path specified. Models will be saved in: "AutogluonModels/ag-20250109_132653"
Preset alias specified: 'medium' maps to 'medium_quality'.
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.2
Python Version:     3.12.2
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #135~20.04.1-Ubuntu SMP Mon Oct 7 13:56:22 UTC 2024
CPU Count:          16
Memory Avail:       3.32 GB / 31.09 GB (10.7%)
Disk Space Avail:   297.46 GB / 693.60 GB (42.9%)
Presets specified: ['medium']
Beginning AutoGluon training ...
AutoGluon will save models to "/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250109_132653"
Train Data Rows:    1199
Train Data Columns: 19
Label Column:       Output of simulation (execution): Total No. of near-misses for AVs (V1-V3)
Problem Type:       regression
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Me

column='Output of simulation (execution): Total No. of near-misses for AVs (V1-V3)':


	0.3113	 = Validation score   (r2)
	0.82s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: RandomForest ...
	0.3562	 = Validation score   (r2)
	0.48s	 = Training   runtime
	0.04s	 = Validation runtime
Fitting model: ExtraTrees ...
	0.384	 = Validation score   (r2)
	2.92s	 = Training   runtime
	0.03s	 = Validation runtime
Fitting model: XGBoost ...
	0.3911	 = Validation score   (r2)
	0.5s	 = Training   runtime
	0.0s	 = Validation runtime
Fitting model: WeightedEnsemble_L2 ...
	Ensemble Weights: {'XGBoost': 0.588, 'ExtraTrees': 0.412}
	0.3977	 = Validation score   (r2)
	0.05s	 = Training   runtime
	0.0s	 = Validation runtime
AutoGluon training complete, total runtime = 4.98s ... Best model: WeightedEnsemble_L2 | Estimated inference throughput: 6462.1 rows/s (240 batch size)
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("/home/olga/coordination_oru/scenario-analysis/AutogluonModels/ag-20250109_132653")


,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,ExtraTrees,0.440215,0.383984,r2,0.046503,0.034244,2.923515,0.046503,0.034244,2.923515,1,True,3
1,RandomForest,0.416142,0.356168,r2,0.042769,0.035512,0.479509,0.042769,0.035512,0.479509,1,True,2
2,WeightedEnsemble_L2,0.412148,0.397695,r2,0.055080,0.037140,3.473104,0.001439,0.000660,0.045642,2,True,5
3,XGBoost,0.364095,0.391135,r2,0.007138,0.002235,0.503947,0.007138,0.002235,0.503947,1,True,4
4,LightGBM,0.342803,0.311297,r2,0.004300,0.002229,0.822900,0.004300,0.002229,0.822900,1,True,1


,Output of simulation (execution): No. of completed missions for AV (V1),Output of simulation (execution): No. of completed missions for AV (V2),Output of simulation (execution): No. of completed missions for AV (V3),Output of simulation (execution): Total No. of completed missions for AVs (V1-V3),Output of simulation (execution): No. of collisions for AV (V1),Output of simulation (execution): No. of collisions for AV (V2),Output of simulation (execution): No. of collisions for AV (V3),Output of simulation (execution): Total No. of collisions for AVs (V1-V3),Output of simulation (execution): No. of near-misses for AV (V1),Output of simulation (execution): No. of near-misses for AV (V2),Output of simulation (execution): No. of near-misses for AV (V3),Output of simulation (execution): Total No. of near-misses for AVs (V1-V3)
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario10-1.json, passhum 0, slowness no, forcing no",13.871549,14.783334,5.817646,33.817295,0.025461,0.003109,0.021403,0.005246,0.016025,0.000000,0.026821,0.013486
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-8.json, passhum 0, slowness with rerouting, forcing change of priorities",8.914466,9.696667,5.099592,24.185030,0.045242,0.110212,-0.063403,0.080016,0.008738,0.006667,0.026821,0.046935
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario5-2.json, passhum 0, slowness no, forcing change of priorities",7.047144,4.186666,3.122889,14.484947,0.569713,1.804756,4.243649,8.183413,0.018022,0.102222,0.033952,0.142977
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario3-6.json, passhum 0, slowness without rerouting, forcing no",4.844720,7.873333,7.872253,20.728573,0.022754,0.010271,-0.026449,-0.003551,0.003130,0.000556,0.026821,0.006855
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-1.json, passhum 0, slowness no, forcing stops",11.697506,14.430000,4.596289,30.431721,2.523361,0.681561,0.579906,5.004415,0.496621,0.726111,0.026821,1.487437
...,...,...,...,...,...,...,...,...,...,...,...,...
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario8-7.json, passhum 0, slowness with rerouting, forcing no",4.154019,4.330000,4.073736,13.302669,0.042322,0.012567,-0.041250,0.007442,-0.007425,0.002778,0.026821,-0.011594
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario10-6.json, passhum 0, slowness no, forcing stops",13.611521,15.980000,8.892031,38.442520,1.984801,0.817333,0.675555,4.366377,3.268807,0.791667,0.054498,3.347152
"map-generator/generated-maps/2024-11-28_13:19:18_without_bridges/scenario3-8.json, passhum 0, slowness no, forcing stops",0.579708,1.486667,2.004342,3.122782,1.594161,0.337037,0.419474,2.503802,0.092067,0.025556,0.026821,0.235089
"map-generator/generated-maps/2024-11-28_13:17:39_with_bridges/scenario1-10.json, passhum 0, slowness with rerouting, forcing change of priorities",20.617020,9.430000,7.270045,38.160870,0.010449,-0.030561,-0.079105,0.005585,0.023670,0.025000,0.026821,0.067047


## evaluate_and_plot_all_columns

In [12]:
evaluate_and_plot_all_columns(df_test, df_predictions_autogluon)

No. of completed missions for AV (V1):
- R^2 Score: 0.8387353420257568


No. of completed missions for AV (V2):
- R^2 Score: 0.7499943375587463


No. of completed missions for AV (V3):
- R^2 Score: 0.9055327773094177


Total No. of completed missions for AVs (V1-V3):
- R^2 Score: 0.8089451193809509


No. of collisions for AV (V1):
- R^2 Score: 0.6572251319885254


...

Total No. of collisions for AVs (V1-V3):
- R^2 Score: 0.5993283987045288


No. of near-misses for AV (V1):
- R^2 Score: 0.40909695625305176


No. of near-misses for AV (V2):
- R^2 Score: 0.30538707971572876


No. of near-misses for AV (V3):
- R^2 Score: 0.0342259407043457


Total No. of near-misses for AVs (V1-V3):
- R^2 Score: 0.41214805841445923


## explain_predictions

In [13]:
def explain_predictions(predictors, df_test):
    # X_test, y_test = split_df_to_X_y(df_test)
    
    for column, predictor in zip(COLUMNS_OUTPUT, predictors):
        # df_test_predictor = pd.concat([X_test, y_test[[column]]], axis=1)
        
        tree_model = predictor._trainer.load_model('LightGBM')
        tree_importance = tree_model.model.feature_importance(importance_type='gain')
        # show(tree_importance, column)
        
        feature_names = predictor.feature_metadata.get_features()
        df = pd.DataFrame.from_dict(
            {name: {'importance': value} for name, value in zip(feature_names, tree_importance)},
            orient='index'
        )
        # print(df.index)
        # Group indexes by their base name before `#`
        df['group'] = df.index.str.extract(r'^(.+?)(?:#\d+)?$', expand=False)
        df = df.groupby('group')['importance'].sum().to_frame()
        
        df.sort_values(by='importance', ascending=False, inplace=True)
        show(df, column)
        
        
explain_predictions(predictors, df_test)

,importance
group,
Output of simulation (planning): Mission length for AV (V1),109185.940323
Output of simulation (planning): CSD score for AV (V1),49849.365924
Violation type: Priority violation,6538.441830
Static map features: No. of OPs,5769.244556
Output of simulation (planning): Total Mission length for AVs (V1-V3),5655.155615
Violation type: Both,5361.533866
Output of simulation (planning): Mission length for MV (V0),4471.854873
Output of simulation (planning): Mission length for AV (V2),4357.377784
Output of simulation (planning): CSD score for AV (V2),3571.151848


,importance
group,
Output of simulation (planning): Mean CSD score for AVs (V1-V3),26119.429463
Output of simulation (planning): Mission length for AV (V2),25489.416263
Violation type: Priority violation,8418.968035
Output of simulation (planning): CSD score for AV (V2),3934.654646
Output of simulation (planning): Mission length for AV (V1),3858.370238
Violation type: Both,3548.234756
Output of simulation (planning): CSD score for AV (V1),3368.286389
Output of simulation (planning): Total Mission length for AVs (V1-V3),2913.661423
Output of simulation (planning): CSD score for MV (V0),2305.513194


,importance
group,
Output of simulation (planning): Mission length for AV (V3),41847.717868
Output of simulation (planning): CSD score for AV (V3),12238.719636
Coordination strategy: Stops,4020.537197
Violation type: Priority violation,3266.774744
Output of simulation (planning): Mission length for AV (V1),2737.617563
Output of simulation (planning): Mission length for MV (V0),2109.502193
Output of simulation (planning): CSD score for MV (V0),2011.071974
Violation type: Both,1622.745627
Output of simulation (planning): Total Mission length for AVs (V1-V3),1516.006576


,importance
group,
Static map features: No. of OPs,315108.891623
Output of simulation (planning): Total Mission length for AVs (V1-V3),55597.920600
Output of simulation (planning): Mission length for AV (V1),54672.345400
Output of simulation (planning): CSD score for AV (V1),52859.357122
Output of simulation (planning): Mission length for AV (V2),37679.368555
Violation type: Priority violation,32064.169384
Violation type: Both,26744.345208
Output of simulation (planning): Mission length for AV (V3),18195.497837
Output of simulation (planning): CSD score for AV (V3),17639.598649


,importance
group,
Violation type: Priority violation,5027.259868
Output of simulation (planning): Mission length for MV (V0),2317.754076
Output of simulation (planning): CSD score for AV (V1),2031.808741
Output of simulation (planning): CSD score for MV (V0),1638.963531
Output of simulation (planning): CSD score for AV (V3),925.181018
Output of simulation (planning): Mission length for AV (V2),464.673681
Output of simulation (planning): Total Mission length for AVs (V1-V3),451.489799
Output of simulation (planning): CSD score for AV (V2),437.644934
Output of simulation (planning): Mean CSD score for AVs (V1-V3),425.985915


...

,importance
group,
Violation type: Priority violation,19528.218926
Output of simulation (planning): Mission length for MV (V0),5375.046431
Coordination strategy: Change of priorities,4263.050934
Output of simulation (planning): CSD score for AV (V1),3772.012009
Output of simulation (planning): Total Mission length for AVs (V1-V3),3531.720995
Output of simulation (planning): Mission length for AV (V2),3134.969155
Output of simulation (planning): CSD score for AV (V3),3085.822727
Output of simulation (planning): CSD score for MV (V0),2702.655164
Output of simulation (planning): CSD score for AV (V2),2609.340639


,importance
group,
Output of simulation (planning): CSD score for AV (V3),271.209641
Output of simulation (planning): CSD score for MV (V0),186.328087
Output of simulation (planning): Mean CSD score for AVs (V1-V3),153.116986
Violation type: Priority violation,140.383539
Output of simulation (planning): Mission length for AV (V1),135.753560
Output of simulation (planning): Mission length for AV (V3),98.435138
Output of simulation (planning): CSD score for AV (V2),87.057669
Output of simulation (planning): Total Mission length for AVs (V1-V3),62.892434
Output of simulation (planning): CSD score for AV (V1),60.002809


,importance
group,
Output of simulation (planning): Mean CSD score for AVs (V1-V3),258.235829
Output of simulation (planning): CSD score for MV (V0),191.838736
Output of simulation (planning): CSD score for AV (V3),186.561041
Output of simulation (planning): Mission length for AV (V1),104.638110
Output of simulation (planning): CSD score for AV (V1),100.359271
Output of simulation (planning): CSD score for AV (V2),80.885005
Violation type: Priority violation,49.987068
Output of simulation (planning): Total Mission length for AVs (V1-V3),45.624324
Output of simulation (planning): Mission length for AV (V2),42.943161


,importance
group,
Output of simulation (planning): Mission length for AV (V3),8.618967e+00
Violation type: Priority violation,5.252330e+00
Output of simulation (planning): CSD score for AV (V1),2.911521e+00
Output of simulation (planning): CSD score for AV (V3),1.016653e+00
Output of simulation (planning): Mission length for AV (V2),9.044310e-01
Output of simulation (planning): CSD score for MV (V0),3.788940e-01
Output of simulation (planning): Total Mission length for AVs (V1-V3),2.338768e-01
Violation type: Both,1.164830e-01
Static map features: Connectivity,4.022570e-02


,importance
group,
Output of simulation (planning): Mean CSD score for AVs (V1-V3),863.950141
Output of simulation (planning): Mission length for AV (V1),661.733091
Output of simulation (planning): CSD score for MV (V0),644.817167
Violation type: Priority violation,438.745137
Output of simulation (planning): CSD score for AV (V3),437.149304
Output of simulation (planning): Mission length for MV (V0),250.298934
Output of simulation (planning): CSD score for AV (V2),222.916342
Output of simulation (planning): CSD score for AV (V1),193.841424
Output of simulation (planning): Mission length for AV (V3),161.645415
